In [2]:
import shapefile

In [3]:
sf = shapefile.Reader("world/data/VGD-Oesterreich_gen_250/BEV_VGD_250_LAM_mitAttributen_shp_2016_10_02.shp")

In [4]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [5]:
shaperecs = sf.shapeRecords()

In [14]:
from django.contrib.gis.geos import Polygon, MultiPolygon

In [41]:
shaperecs[0].shape.points

[(413199.7849999964, 512448.1479999982),
 (413255.9849999994, 512448.52600000054),
 (413254.49099999666, 512609.4059999995),
 (413110.93299999833, 513015.9439999983),
 (413423.5319999978, 512970.9210000001),
 (413642.3189999983, 513176.21400000155),
 (413929.11100000143, 513061.07499999925),
 (413896.186999999, 513326.1160000004),
 (413792.58200000226, 513469.3740000017),
 (414211.16099999845, 513914.4439999983),
 (414306.7950000018, 514206.60599999875),
 (414746.18599999696, 514256.3189999983),
 (414904.5329999998, 514163.7800000012),
 (415278.0719999969, 514129.01000000164),
 (415654.66099999845, 513888.9780000001),
 (415811.1730000004, 513913.3370000012),
 (415576.01099999994, 513544.73699999973),
 (415727.41600000113, 513388.48900000006),
 (415567.97299999744, 513093.6009999998),
 (415592.58900000155, 512639.60599999875),
 (415398.09799999744, 512468.06700000167),
 (415477.6960000023, 512311.36400000006),
 (415777.50599999726, 512240.53400000185),
 (415837.54100000113, 512147.26000

In [62]:
from datetime import date

In [63]:
src = Source(name='BEV Österreich, VGD', original_url="http://www.bev.gv.at/pls/portal/docs/PAGE/BEV_PORTAL_CONTENT_ALLGEMEIN/0200_PRODUKTE/UNENTGELTLICHE_PRODUKTE_DES_BEV/VGD-Oesterreich_gen_250.zip", downloaded=date.today())

In [64]:
src.save()

/home/vagrant/.venvs/geodjango/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1393: RuntimeWarning: DateTimeField Source.downloaded received a naive datetime (2017-01-13 00:00:00) while time zone support is active.
  RuntimeWarning)



In [66]:
sf_fields = sf.fields

In [69]:
sf_fields.pop(0)

('DeletionFlag', 'C', 1, 0)

In [70]:
sf_fields

[['KG_NR', 'C', 6, 0],
 ['KG', 'C', 50, 0],
 ['MERIDIAN', 'C', 3, 0],
 ['GKZ', 'C', 6, 0],
 ['PG', 'C', 50, 0],
 ['BKZ', 'C', 4, 0],
 ['PB', 'C', 50, 0],
 ['FA_NR', 'C', 3, 0],
 ['FA', 'C', 50, 0],
 ['GB_KZ', 'C', 4, 0],
 ['GB', 'C', 50, 0],
 ['VA_NR', 'C', 3, 0],
 ['VA', 'C', 50, 0],
 ['BL_KZ', 'C', 2, 0],
 ['BL', 'C', 50, 0],
 ['ST_KZ', 'N', 2, 0],
 ['ST', 'C', 50, 0],
 ['FL', 'N', 12, 0]]

In [109]:
errors_shapes = []

In [110]:
for x in shaperecs:
    try:
        p = Polygon(x.shape.points, srid=31287)
    except:
        errors_shapes.append(x)
        continue
    p.transform(4326)
    p2 = MultiPolygon(p)
    lst_legacy = dict()
    for indx, f in enumerate(sf_fields):
        if type(x.record[indx]) == bytes:
            entry = x.record[indx].decode('latin1')
        else:
            entry = str(x.record[indx])
        lst_legacy[f[0]] = entry
    if type(x.record[1]) == bytes:
        name = x.record[1].decode('latin1')
    else:
        name = x.record[1]
    a = Area(geom=p2, name=name, legacy_properties=lst_legacy, source=src)
    a.save()

GEOS_ERROR: IllegalArgumentException: Points of LinearRing do not form a closed linestring



In [113]:
g = Area.objects.get(pk=4965)

In [114]:
g.legacy_properties['BKZ']

'414'

In [112]:
errors_shapes[0].record

['85103',
 'Matrei in Osttirol Land',
 'M31',
 '70717',
 'Matrei in Osttirol',
 '707',
 'Lienz',
 '082',
 b'Kitzb\xfchel Lienz',
 '850',
 'Lienz',
 '85',
 'Lienz',
 '7',
 'Tirol',
 1,
 b'\xd6sterreich',
 275388233]

### Bezirkgrenzen

In [123]:
sf2 = shapefile.Reader("world/data/stat_oesterreich_pol_bezirke/STATISTIK_AUSTRIA_POLBEZ_20160101.shp")

In [116]:
shaperecs2 = sf2.shapeRecords()

In [124]:
sf2.fields

[('DeletionFlag', 'C', 1, 0), ['ID', 'C', 254, 0], ['NAME', 'C', 254, 0]]

In [125]:
sf2_fields = sf2.fields

In [126]:
sf2_fields.pop(0)

('DeletionFlag', 'C', 1, 0)

In [127]:
sf2_fields

[['ID', 'C', 254, 0], ['NAME', 'C', 254, 0]]

In [129]:
src2 = Source(name='Statistik Österreich, Grenzen politische Bezirke', original_url="https://www.data.gv.at/katalog/dataset/stat_gliederung-osterreichs-in-politische-bezirke/resource/fc91c2cd-c1e4-41af-b8e9-02615e258cba", downloaded=date.today())

In [130]:
src2.save()

/home/vagrant/.venvs/geodjango/lib/python3.5/site-packages/django/db/models/fields/__init__.py:1393: RuntimeWarning: DateTimeField Source.downloaded received a naive datetime (2017-01-13 00:00:00) while time zone support is active.
  RuntimeWarning)



In [145]:
errors_shapes2 = []

In [146]:
for indx2, x in enumerate(shaperecs2):
    print('import {}'.format(indx2))
    try:
        p = Polygon(x.shape.points, srid=31287)
    except:
        errors_shapes2.append(x)
        shp_closed = x.shape.points
        shp_closed.append(x.shape.points[0])
        p = Polygon(shp_closed, srid=31287)
    p.transform(4326)
    p2 = MultiPolygon(p)
    lst_legacy = dict()
    for indx, f in enumerate(sf2_fields):
        if type(x.record[indx]) == bytes:
            entry = x.record[indx].decode('latin1')
        else:
            entry = str(x.record[indx])
        lst_legacy[f[0]] = entry
    if type(x.record[1]) == bytes:
        name = x.record[1].decode('latin1')
    else:
        name = x.record[1]
    a = Area(geom=p2, name=name, legacy_properties=lst_legacy, source=src2)
    a.save()

import 0
import 1
import 2
import 3
import 4
import 5
import 6
import 7
import 8
import 9
import 10
import 11
import 12
import 13
import 14
import 15
import 16
import 17
import 18
import 19
import 20
import 21
import 22
import 23
import 24
import 25
import 26
import 27
import 28
import 29
import 30
import 31
import 32
import 33
import 34
import 35
import 36
import 37
import 38
import 39
import 40
import 41
import 42
import 43
import 44
import 45
import 46
import 47
import 48
import 49
import 50
import 51
import 52
import 53
import 54
import 55
import 56
import 57
import 58
import 59
import 60
import 61
import 62
import 63
import 64
import 65
import 66
import 67
import 68
import 69
import 70
import 71
import 72
import 73
import 74
import 75
import 76
import 77
import 78
import 79
import 80
import 81
import 82
import 83
import 84
import 85
import 86
import 87
import 88
import 89
import 90
import 91
import 92
import 93
import 94


In [142]:
shaperecs2[0].shape.points

[(637378.7599999998, 439925.47000000067),
 (637377.1099999994, 439927.1400000006),
 (637314.75, 439990.3499999996),
 (637302.6600000001, 440017.9399999995),
 (637288.3100000005, 440050.6799999997),
 (637203.5399999991, 440119.26999999955),
 (637196.8699999992, 440124.6699999999),
 (637191.6999999993, 440128.8499999996),
 (637179.4499999993, 440138.76999999955),
 (637172.8100000005, 440144.1400000006),
 (637160.6500000004, 440154.8499999996),
 (637158.7699999996, 440156.5),
 (637149.1300000008, 440164.98000000045),
 (637122.0099999998, 440188.8499999996),
 (637112.0899999999, 440197.5800000001),
 (637106.2899999991, 440202.6899999995),
 (637098.8200000003, 440209.2599999998),
 (637096.9199999999, 440210.9399999995),
 (637092.9100000001, 440214.48000000045),
 (637089.5700000003, 440217.44999999925),
 (637077.6699999999, 440228.1899999995),
 (637000.0600000005, 440303.47000000067),
 (636986.1699999999, 440318.83999999985),
 (636922.1600000001, 440389.6799999997),
 (636860.9399999995, 4404